In [225]:
import pandas as pd
import json
import pydicom

### Explore RIS <--> Radiology files (resulting files)

- Total number of rows: 0 to 248 --> 249
- Unique PAT_IDS: 80 <- means there are repeatative patients
- Unique IDS: 248
All patients are Female

##### Issue: 
- Misleading that pat_id has structure: 
as: "bf98f2e9-f02e-48b0-8472-470af8aebf7a" NOT as: "1.2.826.0.1.10184103.5.1.101269"
Tried decode by sha1, sha256, basecode64 - unsuccsessfully
- There repeatative pat_ids. 

#### PAT_IDS {bf98f2e9-f02e-48b0-8472-470af8aebf7a}
- Amount of all pat_ids == 248.
- Amount of unique pat_ids == 80.
- Amount of unique ids == 248.
That means, some of patients had made experiments multiple time. 

    
### Explore PAC <--> DICOM *one line per image 

- Total number of rows: 904 <- it means in totall made 904 experiments or photos of breast 

#Notes
- Per each DICOM image found patient ID
##### PAT_IDS {1.2.826.0.1.10184103.5.1.10126}
- Amount of all pat_id == 904
- Amount of unique pat_id == 290
- Amount of "oddly formed" pat_ids out of unique pat_id == 71
i.e out of 290 unique pat_id 71 ("df70ee3f-8f64-4ab4-af78-a3a739a0a347") is wrongly formed, 219 ("1.2.826.0.1.10184103.5.1.106101") unique record is clean 

##### accession_number
- Amount of all accession_number == 904
- Amount of unique accession_number == 248

##### date_of_studies
- Amount of all date_of_studies == 904
- Amount of unique date_of_studies == 80 <- it means there were 
                                          days when multiple patients made exams

##### Issue: 
- Misleading that 71 pat_id has structure: 
as: "bf98f2e9-f02e-48b0-8472-470af8aebf7a" NOT as "1.2.826.0.1.10184103.5.1.101269" 
Tried decode by sha1, sha256, basecode64 - unsuccsessfully
- There repeatative pat_ids. 

In [457]:
# read ris data
ris_data = pd.read_csv('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/ris.csv')

# ris unique_patients_ids
unique_ris_patients_ids = ris_data['pat_id'].unique()
ris_accession_number = ris_data['id']

# ris_data[ris_data['id'] == '5cd192a0-fcb0-4fb8-8f29-a5f88afa7256']

# print(len(pacs_accession_number))
# print(len(ris_accession_number))

# count = 0
# for pac_access_id in pacs_accession_number:
#     for ris_access_id in ris_accession_number:
#         if pac_access_id == ris_access_id:
#             print(f'PAC access: {pac_access_id};  RIS access {ris_access_id}')
#             count += 1
# count

# for date in ris_data['pat_dob']:
#     date_datetime = datetime.strptime(str(date), '%Y%M%d')
#     print(f'{date_datetime.year}.{date_datetime.month}.{date_datetime.day}')

# # side, date, opinion
# columns = ['outcome_l', 'outcome_r']

# opinion = ris_data[ris_data['id'] == '41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0'][columns]
# opinion_1 = opinion.iloc[0]['outcome_l']
# opinion_2 = opinion.iloc[0]['outcome_r']


ris_data[ris_data['id'] == '41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0']['date'].item()

# date_datetime = datetime.strptime(str(ris_data['date'][0]), '%Y%M%d')

# print(f'{date_datetime.year}.{date_datetime.month}.{date_datetime.day}')


20140808

In [288]:
# Try creare DataFrame from dicom json 
from pydicom.dataset import Dataset

with open('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/pacs.json.csv', 'r') as f:
    pac_file = f.readlines()

pac_df_from_json = Dataset.from_json(pac_file[0])

pac_df_from_json.PatientBirthDate

#['PatientBirthDate', 'PatientID', 'PatientName', 'PatientSetupSequence', 'PatientSex']


# pydicom.charset.encode_string(value, encodings)
# pydicom.charset.convert_encodings(encodings)
#print(pydicom.charset.encode_string(pac_df_from_json.PatientID))

# print(pydicom.charset.encode_string(pac_df_from_json.PatientID,
#                                     pydicom.charset.convert_encodings(pac_df_from_json.PatientID),
#                                     ))


'18880401'

In [222]:
# read pacs data
with open('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/pacs.json.csv', 'r') as f:
    pac_file = f.readlines()


# convert string representation of dict to dict    
#https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary

# pac unique_patients_ids

pacs_patients_ids = []
pacs_accession_number = []
date_of_studies = []

for patient in pac_file:
    json_acceptable_string = patient.replace("'", "\"")
    d = json.loads(json_acceptable_string)    

    pacs_patients_ids.append(d['00100020']['Value'][0]) # PAC Patients IDS
    pacs_accession_number.append(d['00080050']['Value'][0]) # PAC Accession number
    date_of_studies.append(d['00080020']['Value'][0]) # PAC Date of Studies 

    
unique_pacs_patients_ids = list(set(pacs_patients_ids)) # 290(all) - 71(odd formed)
unique_pacs_accession_number = list(set(pacs_accession_number)) # 248
unique_date_of_studies = list(set(date_of_studies))

len(unique_pacs_accession_number)

248

In [295]:
count = 0
equal_ids = []

for pac_p in unique_pacs_patients_ids:
    for ris_p in unique_ris_patients_ids:
        if pac_p == ris_p:
            count += 1
            equal_ids.append(pac_p)
            
#equal_ids

### Retrieve "pacs.json.csv" --> DICOM files

From here we will get:
 - patient_uid: 1.2.3.4,
 - sex: F,
 - date_of_birth: 1955.05.14,
 - studies: [...]



In [460]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from pydicom.dataset import Dataset


path_to_pacs = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/pacs.json.csv')
path_to_ris = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/ris.csv')
path_to_lim = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/lims.txt')

# check whether file exisist
assert path_to_pacs.exists(), 'PACS file does not exisist'

# we can read PACS using 2 approaches

### Approach 1 ##################
# Read as usual file
with open(path_to_pacs, 'r') as f:
    pac_data = f.readlines() # List[nested dicts],
                             # where each item of list (904 items) is dedicated examination 

# Iterate trought each examinations
pacs_patients_ids = [] # patient id {1.2.826.0.1.10184103.5.1.80750}, but faced with {bf98f2e9-f02e-48b0-8472-470af8aebf7a}
pacs_accession_number = [] # patient accession number {}
pacs_date_of_studies = [] # date when accession happened {}

for patient_index, patient in enumerate(tqdm(pac_data)):
    json_acceptable_string = patient.replace("'", "\"")
    d = json.loads(json_acceptable_string)    

    pacs_patients_ids.append(d['00100020']['Value'][0]) # PAC Patients IDS
    pacs_accession_number.append(d['00080050']['Value'][0]) # PAC Accession number
    pacs_date_of_studies.append(d['00080020']['Value'][0]) # PAC Date of Studies 
    
unique_pacs_patients_ids = list(set(pacs_patients_ids))


###  MAIN  ##########################################
### Approach 2 ##################
# Uisng PyDicom external lib to convert string representation of json to dicom format
# It allows automatically make mapping for keys to normal dicom format 
# By "Accession Number" key in PAC we can find info for RIS by "ID" key <-- mapping found   

# # Read as usual file
# with open(path_to_pacs, 'r') as f:
#     pac_data = f.readlines() # List[nested dicts],
#                              # where each item of list (904 items) is dedicated examination 


rows = [] 

# Iterate trought each examination
for patient_index, patient in enumerate(tqdm(pac_data)):
    buf_dicom = Dataset.from_json(patient)
    
    # Patient ID 
    patient_id = buf_dicom.PatientID # can be wrong, need check with assert later

    # Patient Gender 
    patient_gender = buf_dicom.PatientSex
    
    # Date of birth
    patient_date_of_birth = buf_dicom.PatientBirthDate
    patient_date_of_birth = datetime.strptime(str(date), '%Y%M%d')
    # Patient date of birth is same for all PATIENTS(BUG IN DATA) 
    ## IT MEANS BETTER TO GREP DATE OF BIRTH FROM "RIS" FILE 
    patient_date_of_birth = f'{date_datetime.year}.{date_datetime.month}.{date_datetime.day}'
    
    # Access number
    patient_access_number = buf_dicom.AccessionNumber
    
    # PAC Studies {DICOM STUDIES}
    StudyID = buf_dicom.StudyID 
    StudyDate = buf_dicom.StudyDate
    StudyTime = buf_dicom.StudyTime
    StudyDescription = buf_dicom.StudyDescription
    StudyInstanceUID = buf_dicom.StudyInstanceUID
    StudyStatusID = buf_dicom.StudyStatusID
    StudyComments = buf_dicom.StudyComments
    
    # RIS DATA {RADIOLOGY STUDIES} side, date, opinion
    # read RIS data
    ris_df = pd.read_csv(path_to_ris)
    columns = ['outcome_l', 'outcome_r']
    
    # retrieve sides 
    opinions = ris_data[ris_data['id'] == patient_access_number][columns]
    voice_1 = opinions.iloc[0]['outcome_l']
    voice_2 = opinions.iloc[0]['outcome_r']
    
    # Date
    ris_date = ris_data[ris_data['id'] == patient_access_number]['date'].item()
    ris_date = datetime.strptime(str(ris_date), '%Y%M%d')
    ris_date = f'{date_datetime.year}.{date_datetime.month}.{date_datetime.day}'
    
    # LIMS DATA {PATHOLOGY STUDIES} opinion, date
    # read LIMS data
    
    
    # Date
    lims_date = 
    
    # Diagnos
    lims_opinion = 
    
    
    buffer_row = {
        'patient_uid': patient_id,
        'sex': patient_gender,
        'date_of_birth': patient_date_of_birth, # should be obtained from RIS file
        'studies': [{'StudyID': StudyID if not len(StudyID) == 0 else np.nan,
                    'StudyDate': StudyDate if not len(StudyDate) == 0 else np.nan,
                    'StudyTime': StudyTime if not len(StudyTime) == 0 else np.nan,
                    'StudyDescription': StudyDescription if not len(StudyDescription) == 0 else np.nan,
                    'StudyInstanceUID': StudyInstanceUID if not len(StudyInstanceUID) == 0 else np.nan,
                    'StudyStatusID': StudyStatusID if not len(StudyStatusID) == 0 else np.nan,
                    'StudyComments': StudyComments if not len(StudyComments) == 0 else np.nan,
                    }],
        'rad': [{
            'side': (voice_1, voice_2),
            'date': ris_date,
            'opinion': voice_1 if voice_1 == voice_2 else (voice_1, voice_2)
        }],
        
        'patho': [{
            'date': ,
            'opinion': 
        }]
#        'record_status': 'Verified' if str(patient_id).startswith('1') else 'Faulty'
    }
    
    rows.append(buffer_row)
    
    
# write results to file             
with open("/Users/macbook/Desktop/result_json.txt", "w") as file:
    file.write(json.dumps(rows))



100%|██████████| 904/904 [00:05<00:00, 151.95it/s]


In [406]:
for patient_index, patient in enumerate(tqdm(pac_data)):
    buf_dicom = Dataset.from_json(patient)
    
    
    
    StudyID = buf_dicom.StudyID 
    StudyDate = buf_dicom.StudyDate
    StudyTime = buf_dicom.StudyTime
    StudyDescription = buf_dicom.StudyDescription
    StudyInstanceUID = buf_dicom.StudyInstanceUID
    StudyStatusID = buf_dicom.StudyStatusID
    StudyComments = buf_dicom.StudyComments
    if len(StudyID)
    print(StudyID, StudyDate, StudyTime, StudyDescription, StudyInstanceUID, StudyStatusID, StudyComments)
    print('#######')
    


  9%|▉         | 85/904 [00:00<00:02, 407.85it/s]

 20221030 000000.000000  1.2.826.0.1.10184103.5.4.175302 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.175302 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.175302 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.175302 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.210987 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.210987 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.210987 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.210987 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.221299 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.221299 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.221299 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.221299 COMPLETED 
#######
 20221030 000000.000000  1.2.826.0.1.10184103.5.4.63382 COMPLETED 
#######
 20221030 000

 13%|█▎        | 117/904 [00:00<00:02, 375.48it/s]

 20190307 000000.000000  1.2.826.0.1.10184103.5.4.268294 COMPLETED 
#######
 20220314 000000.000000  1.2.826.0.1.10184103.5.4.89694 COMPLETED 
#######
 20220314 000000.000000  1.2.826.0.1.10184103.5.4.89694 COMPLETED 
#######
 20220314 000000.000000  1.2.826.0.1.10184103.5.4.89694 COMPLETED 
#######
 20220314 000000.000000  1.2.826.0.1.10184103.5.4.89694 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.175651 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.175651 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.175651 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.175651 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.175651 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.107077 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.107077 COMPLETED 
#######
 20210920 000000.000000  1.2.826.0.1.10184103.5.4.107077 COMPLETED 
#######
 20210920 000000

 23%|██▎       | 207/904 [00:00<00:01, 373.19it/s]

...

 20230224 000000.000000  1.2.826.0.1.10184103.5.4.79933 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.359556 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.359556 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.359556 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.359556 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.207422 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.207422 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.207422 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.207422 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.146417 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.146417 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.146417 COMPLETED 
#######
 20290710 000000.000000  1.2.826.0.1.10184103.5.4.146417 COMPLETED 
#######
 20290710 000

 93%|█████████▎| 843/904 [00:02<00:00, 331.24it/s]

 20240709 000000.000000  1.2.826.0.1.10184103.5.4.22364 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.22364 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.22364 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.22364 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.39927 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.39927 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.39927 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.39927 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.128776 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.128776 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.128776 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.128776 COMPLETED 
#######
 20240709 000000.000000  1.2.826.0.1.10184103.5.4.279809 COMPLETED 
#######
 20240709 000000.000

100%|██████████| 904/904 [00:02<00:00, 310.86it/s]

 20240412 000000.000000  1.2.826.0.1.10184103.5.4.250993 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.203621 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.203621 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.203621 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.203621 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.314270 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.314270 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.314270 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.314270 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.151954 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.151954 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.151954 COMPLETED 
#######
 20221130 000000.000000  1.2.826.0.1.10184103.5.4.151954 COMPLETED 
#######
 20200505 00

In [401]:
# methods = ['AccessionNumber', 'AcquisitionContextSequence', 'AcquisitionDate', 'AcquisitionDeviceProcessingCode', 'AcquisitionDeviceProcessingDescription', 'AcquisitionNumber', 'AcquisitionTime', 'AdditionalPatientHistory', 'AnatomicRegionSequence', 'AnnotationDisplayFormatID', 'AnodeTargetMaterial', 'BitsAllocated', 'BitsStored', 'BodyPartExamined', 'BodyPartThickness', 'BorderDensity', 'BreastImplantPresent', 'BurnedInAnnotation', 'Columns', 'CompressionForce', 'ContentDate', 'ContentTime', 'ContrastBolusAgent', 'CurrentPatientLocation', 'DateOfLastDetectorCalibration', 'DerivationCodeSequence', 'DerivationDescription', 'DetectorConfiguration', 'DetectorID', 'DetectorType', 'DeviceSerialNumber', 'DistanceSourceToDetector', 'EntranceDose', 'EntranceDoseInmGy', 'EstimatedRadiographicMagnificationFactor', 'EthnicGroup', 'Exposure', 'ExposureControlMode', 'ExposureControlModeDescription', 'ExposureInuAs', 'ExposureTime', 'FieldOfViewDimensions', 'FieldOfViewShape', 'FileMetaInformationVersion', 'FilmOrientation', 'FilterMaterial', 'GantryID', 'Grid', 'HighBit', 'ImageBoxPosition', 'ImageDisplayFormat', 'ImageLaterality', 'ImageType', 'ImagerPixelSpacing', 'ImplementationClassUID', 'ImplementationVersionName', 'InstanceNumber', 'InstitutionAddress', 'InstitutionName', 'IssuerOfPatientID', 'KVP', 'Laterality', 'LossyImageCompression', 'Manufacturer', 'MediaStorageSOPClassUID', 'MediaStorageSOPInstanceUID', 'Modality', 'OperatorsName', 'OrganDose', 'OrganExposed', 'OtherPatientIDs', 'PartialView', 'PatientAge', 'PatientBirthDate', 'PatientComments', 'PatientID', 'PatientName', 'PatientOrientation', 'PatientSex', 'PerformedProcedureStepDescription', 'PerformedProcedureStepID', 'PerformedProcedureStepStartDate', 'PerformedProcedureStepStartTime', 'PhotometricInterpretation', 'PixelData', 'PixelIntensityRelationship', 'PixelIntensityRelationshipSign', 'PixelRepresentation', 'PixelSpacing', 'PositionerPrimaryAngle', 'PositionerType', 'PresentationIntentType', 'PresentationLUTShape', 'ReasonForStudy', 'ReferencedPatientSequence', 'ReferencedStudySequence', 'ReferringPhysicianName', 'RelativeXRayExposure', 'RequestAttributesSequence', 'RequestedProcedureID', 'RequestingPhysician', 'RequestingService', 'RescaleIntercept', 'RescaleSlope', 'RescaleType', 'Rows', 'SOPClassUID', 'SOPInstanceUID', 'SamplesPerPixel', 'Sensitivity', 'SeriesDate', 'SeriesInstanceUID', 'SeriesNumber', 'SeriesTime', 'SoftwareVersions', 'SourceApplicationEntityTitle', 'SourceImageSequence', 'SpecificCharacterSet', 'StationName', 'StudyComments', 'StudyDate', 'StudyDescription', 'StudyID', 'StudyInstanceUID', 'StudyStatusID', 'StudyTime', 'TransferSyntaxUID', 'Trim', 'VOILUTFunction', 'ViewCodeSequence', 'WindowCenter', 'WindowWidth', 'XRayTubeCurrent', '__contains__', '__delattr__', '__delitem__', '__dir__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_character_set', '_convert_pixel_data_using_handler', '_convert_pixel_data_without_handler', '_dataset_slice', '_do_pixel_data_conversion', '_pretty_str', '_set_file_meta', '_slice_dataset', 'add', 'add_new', 'clear', 'convert_pixel_data', 'copy', 'data_element', 'decode', 'decompress', 'dir', 'elements', 'ensure_file_meta', 'fix_meta_info', 'formatted_lines', 'from_json', 'fromkeys', 'get', 'get_item', 'get_private_item', 'group_dataset', 'is_original_encoding', 'items', 'iterall', 'keys', 'overlay_array', 'pixel_array', 'pop', 'popitem', 'private_block', 'private_creators', 'remove_private_tags', 'save_as', 'set_original_encoding', 'setdefault', 'to_json', 'to_json_dict', 'top', 'trait_names', 'update', 'values', 'walk']

# methods

In [528]:
# retrieve LIM file

path_to_lim = Path('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/lims.txt')


def process_file(fpath, logger):
    """Process an HL7 file with the given fpath.
    Returns: an array with all the messages
    """
    message = []
    for line in open(fpath):
        line = line.rstrip('\n')
        line = line.strip()
        if line[:3] in ['FHS', 'BHS', 'FTS', 'BTS']:
            continue
        if line[:3] == 'MSH':
            if message:
                yield message
            message = [line]
        else:
            if len(message) == 0:
                logger.error(
                    'Segment received before message header [%s]',
                    line)
                continue
            if line:
                message.append(line)


if __name__ == '__main__':
    import argparse
    import logging
    import pprint

    # Setup logging
    handlers = []
    handlers.append(logging.StreamHandler())
    logging.basicConfig(
        level=logging.DEBUG,
        format='%(asctime)s:%(levelname)s:%(message)s',
        handlers=handlers)

    # Setup argument parsing
    parser = argparse.ArgumentParser(description='Process HL7 file')

#     parser.add_argument(
#         'fpath',
#         metavar='hl7_file_path',
#         type=str,
#         help='path to the input HL7 file')

#     args = parser.parse_args()
#     fpath = args.fpath
    parser = argparse.Namespace(
        fpath=path_to_lim
    )
    
    parser = vars(parser)
    
    logging.info('Start processing file {}'.format(parser['fpath']))
    results = process_file(parser['fpath'], logging)
    
    lims_ids = []
    for index, result in enumerate(results):
        print(f'OBSERVATION № {index}')
        for item in result:
            print(item)
            if item.startswith('PID'):
                lims_ids.append(item.split('|||')[1][3:]) # correct 

    # pprint.pprint([res for res in result])
    logging.info('Finished processing file {}'.format(parser['fpath']))
    
lims_ids    


2020-08-14 20:11:50,974:INFO:Start processing file /Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/lims.txt
2020-08-14 20:11:50,976:INFO:Finished processing file /Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/lims.txt


OBSERVATION № 0
MSH|^~\&|TEST||KHEIRON||202007010322||ADT^Z03|TEST-95645|P|2.3|||NE|AL|
EVN|Z03|202007010322||01|api^Api user|
PID|||MVZ6bb3b35a-749a-4aa0-998a-77145b176cf8|||1963|F|||||||||||||
OBR|||ef0189ad-b412-4db9-a4d2-152ac3e2ad94||||20190422||||||||||||||||20190422||||F|||||||idXXX
OBX||ST|RESDIAG^Result diagnosis:||C5012^|||
OBSERVATION № 1
MSH|^~\&|TEST||KHEIRON||202007010322||ADT^Z03|TEST-95645|P|2.3|||NE|AL|
EVN|Z03|202007010322||01|api^Api user|
PID|||MVZ9dd13293-ce18-4f1e-8d25-64dafb2bfd7f|||1962|F|||||||||||||
OBR|||3ea1898a-046a-434a-bb0c-772b70b445a9||||20220324||||||||||||||||20220324||||F|||||||idXXX
OBX||ST|RESDIAG^Result diagnosis:||C5012^|||
OBSERVATION № 2
MSH|^~\&|TEST||KHEIRON||202007010322||ADT^Z03|TEST-95645|P|2.3|||NE|AL|
EVN|Z03|202007010322||01|api^Api user|
PID|||MVZa45931c9-b895-4b49-bf25-a7ac5cc81754|||1970|F|||||||||||||
OBR|||1d53a30a-79b7-4949-aee7-54666a8fd820||||20220425||||||||||||||||20220425||||F|||||||idXXX
OBX||ST|RESDIAG^Result diagnosis:||C5

['6bb3b35a-749a-4aa0-998a-77145b176cf8',
 '9dd13293-ce18-4f1e-8d25-64dafb2bfd7f',
 'a45931c9-b895-4b49-bf25-a7ac5cc81754',
 '54e4035a-b9eb-4118-ba1e-635e3f934c8a',
 'b4dcdc27-b876-4ee1-a24d-3d14472f96ee']

In [679]:
### Function wich handle LIMS file
### Returns list of external PAT_IDS + write down to file parsered data. 


count = 0
separator = '-'
rows = []
lims_patients_external_ids = []

for line in open('/Users/macbook/Documents/GitRep/PracticesForEngineers/Practices-for-Engineers/CompaniesPractices/Company10/samples/lims.txt'):        
    cleaned_line = re.sub('[\W_]+', ' ', line).split()
   # assert cleaned_line[0] in ['MSH', 'EVN', 'PID', 'OBR', 'OBX'], 'Wrong key detected.'
    print(cleaned_line)
    if len(cleaned_line) == 0:
        print('FOund blank line')
        continue
    # work with PID {Personal Patient Information} key
    if cleaned_line[0] == 'PID':
        print('PROCESSED PID')
        assert len(cleaned_line) == 8, 'Wrong len of PID record.'
          
        lims_patient_external_id = (separator.join(cleaned_line[1:6]))[3:]
        lims_patient_internal_id = separator.join(cleaned_line[1:6])
        
        lims_patients_external_ids.append(lims_patient_external_id)
        
        print(lims_patient_external_id)
        print(lims_patient_internal_id)
        
    if cleaned_line[0] == 'OBR': # work with OBR key
        print('PROCESSED OBR')
        if len(cleaned_line) != 10:
            print('Found OBR record with missing ID.')
            # ID
            lims_observation_request_id = np.nan
        
            # requested_date
            lims_requested_date = cleaned_line[1]
            lims_requested_date = datetime.strptime(lims_requested_date, '%Y%M%d')
            lims_requested_date = f'{lims_requested_date.year}.{lims_requested_date.month}.{lims_requested_date.day}'
        
            # observation_date
            lims_observation_date = cleaned_line[2]
            lims_observation_date = datetime.strptime(lims_observation_date, '%Y%M%d')
            lims_observation_date = f'{lims_observation_date.year}.{lims_observation_date.month}.{lims_observation_date.day}'
        
            # observation_status
            lims_observation_status = cleaned_line[3]
            
        else:
            # ID
            lims_observation_request_id = np.nan
        
            # requested_date
            lims_requested_date = cleaned_line[6]
            lims_requested_date = datetime.strptime(lims_requested_date, '%Y%M%d')
            lims_requested_date = f'{lims_requested_date.year}.{lims_requested_date.month}.{lims_requested_date.day}'
        
            # observation_date
            lims_observation_date = cleaned_line[7]
            lims_observation_date = datetime.strptime(lims_observation_date, '%Y%M%d')
            lims_observation_date = f'{lims_observation_date.year}.{lims_observation_date.month}.{lims_observation_date.day}'
        
            # observation_status
            lims_observation_status = cleaned_line[8]
        
        print(lims_observation_request_id)
        print(lims_requested_date)
        print(lims_observation_date)
        print(lims_observation_status)
        
        
    if cleaned_line[0] == 'OBX': # work with OBX key
        print('PROCESSED OBX')
        assert len(cleaned_line) == 6, 'Wrong len of OBR record.'
        
#         lims_observation_sub_id = # if empty --> same as lims_observation_request_id
#         lims_observation_value = # RES DIAG

        lims_observation_result_status = str(cleaned_line[5][0])
        lims_observation_probability = int(cleaned_line[5][1:]) / 100
        lims_result_status_repr = 'Record coming over is a correction and thus replaces a final result' 
        # if C --> Record coming over is a correction and thus replaces a final result
        # if D --> Deletes the OBX record
        # if F --> Final results; Can only be changed with a corrected result.
        # if I --> Specimen in lab; results pending
        # if P --> Preliminary results
        # if R --> Results entered -- not verified
        # if S --> Partial results
        # if U --> Results status change to final without retransmitting results already sent as ‘preliminary.’  E.g., radiology changes status from preliminary to final
        # if W --> Post original as wrong, e.g., transmitted for wrong patient
        # if X --> Results cannot be obtained for this observation
        print(lims_observation_result_status)
        print(lims_observation_probability)
        print(lims_result_status_repr)
        
    count += 1
    
    if count == 5:  
        print('Adding to buff dict to main dict.')
        buffer_rows = {
            'MSH': np.nan,
            'ENV': np.nan,
            'PID': [{
                'external_pat_id': lims_patient_external_id,
                'internal_pat_id': lims_patient_internal_id
                }],

            'OBR': [{
                'observation_request_id': lims_observation_request_id,
                'observation_requested_date': lims_requested_date,
                'observation_date': lims_observation_date,
                'observation_status': lims_observation_status
            }],
            
            'OBX': [{
                'observation_probability': lims_observation_probability,
                'observation_result_status': lims_observation_result_status,
                'observation_result_status_representation': lims_result_status_repr if lims_observation_result_status == 'C' else 'other'
            }]
        } 
        rows.append(buffer_rows)
        count = 0
    
# write results to file             
with open("/Users/macbook/Desktop/lims_json.txt", "w") as file:
    file.write(json.dumps(rows))

['MSH', 'TEST', 'KHEIRON', '202007010322', 'ADT', 'Z03', 'TEST', '95645', 'P', '2', '3', 'NE', 'AL']
['EVN', 'Z03', '202007010322', '01', 'api', 'Api', 'user']
['PID', 'MVZ6bb3b35a', '749a', '4aa0', '998a', '77145b176cf8', '1963', 'F']
PROCESSED PID
6bb3b35a-749a-4aa0-998a-77145b176cf8
MVZ6bb3b35a-749a-4aa0-998a-77145b176cf8
['OBR', 'ef0189ad', 'b412', '4db9', 'a4d2', '152ac3e2ad94', '20190422', '20190422', 'F', 'idXXX']
PROCESSED OBR
nan
2019.1.22
2019.1.22
F
['OBX', 'ST', 'RESDIAG', 'Result', 'diagnosis', 'C5012']
PROCESSED OBX
C
50.12
Record coming over is a correction and thus replaces a final result
Adding to buff dict to main dict.
[]
FOund blank line
['MSH', 'TEST', 'KHEIRON', '202007010322', 'ADT', 'Z03', 'TEST', '95645', 'P', '2', '3', 'NE', 'AL']
['EVN', 'Z03', '202007010322', '01', 'api', 'Api', 'user']
['PID', 'MVZ9dd13293', 'ce18', '4f1e', '8d25', '64dafb2bfd7f', '1962', 'F']
PROCESSED PID
9dd13293-ce18-4f1e-8d25-64dafb2bfd7f
MVZ9dd13293-ce18-4f1e-8d25-64dafb2bfd7f
['OBR',

In [682]:
lims_patients_external_ids

['6bb3b35a-749a-4aa0-998a-77145b176cf8',
 '9dd13293-ce18-4f1e-8d25-64dafb2bfd7f',
 'a45931c9-b895-4b49-bf25-a7ac5cc81754',
 '54e4035a-b9eb-4118-ba1e-635e3f934c8a',
 'b4dcdc27-b876-4ee1-a24d-3d14472f96ee',
 'df70ee3f-8f64-4ab4-af78-a3a739a0a347']

In [707]:
set(pd.Series(pacs_accession_number).unique()) == set(ris_df['id']) # unique PAC patients


True

In [684]:
count = 0
for ris_id in ris_data['pat_id'].unique():
    for lim_id in lims_ids:
        if ris_id == lim_id:
            print('found')
            print(ris_id, lim_id)
            count += 1          

found
6bb3b35a-749a-4aa0-998a-77145b176cf8 6bb3b35a-749a-4aa0-998a-77145b176cf8
found
9dd13293-ce18-4f1e-8d25-64dafb2bfd7f 9dd13293-ce18-4f1e-8d25-64dafb2bfd7f
found
a45931c9-b895-4b49-bf25-a7ac5cc81754 a45931c9-b895-4b49-bf25-a7ac5cc81754
found
54e4035a-b9eb-4118-ba1e-635e3f934c8a 54e4035a-b9eb-4118-ba1e-635e3f934c8a
found
b4dcdc27-b876-4ee1-a24d-3d14472f96ee b4dcdc27-b876-4ee1-a24d-3d14472f96ee


In [517]:
pac_patients_ids = []
for patient_index, patient in enumerate(tqdm(pac_data)):
    buf_dicom = Dataset.from_json(patient)
    
    # Patient ID 
    patient_id = buf_dicom.PatientID # can be wrong, need check with assert later
    pac_patients_ids.append(patient_id)

count = 0
for pac_id in pac_patients_ids:
    for lim_id in lims_ids:
        if pac_id == lim_id:
            count += 1
            print('found')
            print(pac_id, lim_id)
            
count

100%|██████████| 904/904 [00:01<00:00, 535.53it/s]

found
6bb3b35a-749a-4aa0-998a-77145b176cf8 6bb3b35a-749a-4aa0-998a-77145b176cf8
found
6bb3b35a-749a-4aa0-998a-77145b176cf8 6bb3b35a-749a-4aa0-998a-77145b176cf8
found
6bb3b35a-749a-4aa0-998a-77145b176cf8 6bb3b35a-749a-4aa0-998a-77145b176cf8
found
6bb3b35a-749a-4aa0-998a-77145b176cf8 6bb3b35a-749a-4aa0-998a-77145b176cf8
found
6bb3b35a-749a-4aa0-998a-77145b176cf8 6bb3b35a-749a-4aa0-998a-77145b176cf8
found
9dd13293-ce18-4f1e-8d25-64dafb2bfd7f 9dd13293-ce18-4f1e-8d25-64dafb2bfd7f
found
9dd13293-ce18-4f1e-8d25-64dafb2bfd7f 9dd13293-ce18-4f1e-8d25-64dafb2bfd7f
found
a45931c9-b895-4b49-bf25-a7ac5cc81754 a45931c9-b895-4b49-bf25-a7ac5cc81754
found
a45931c9-b895-4b49-bf25-a7ac5cc81754 a45931c9-b895-4b49-bf25-a7ac5cc81754
found
a45931c9-b895-4b49-bf25-a7ac5cc81754 a45931c9-b895-4b49-bf25-a7ac5cc81754
found
a45931c9-b895-4b49-bf25-a7ac5cc81754 a45931c9-b895-4b49-bf25-a7ac5cc81754
found
a45931c9-b895-4b49-bf25-a7ac5cc81754 a45931c9-b895-4b49-bf25-a7ac5cc81754
found
54e4035a-b9eb-4118-ba1e-635e3f934c

23

In [790]:
rows = []

for patient_index, patient in enumerate(tqdm(pac_data)):
    buf_dicom = Dataset.from_json(patient)
    
    if patient_index == 9:
        wrong_df = Dataset.from_json(patient)
        
    # Patient ID 
    patient_id = buf_dicom.PatientID # can be wrong, need check with assert later

    # Patient Gender 
    patient_gender = buf_dicom.PatientSex
    
    # Date of birth
    patient_date_of_birth = buf_dicom.PatientBirthDate
    patient_date_of_birth = datetime.strptime(str(date), '%Y%M%d')
    # Patient date of birth is same for all PATIENTS(BUG IN DATA) 
    ## IT MEANS BETTER TO GREP DATE OF BIRTH FROM "RIS" FILE 
    patient_date_of_birth = f'{date_datetime.year}.{date_datetime.month}.{date_datetime.day}'
    
    # Access number
    patient_access_number = buf_dicom.AccessionNumber
    
    # PAC Studies {DICOM STUDIES}
    StudyID = buf_dicom.StudyID 
    StudyDate = buf_dicom.StudyDate
    StudyTime = buf_dicom.StudyTime
    StudyDescription = buf_dicom.StudyDescription
    StudyInstanceUID = buf_dicom.StudyInstanceUID
    StudyStatusID = buf_dicom.StudyStatusID
    StudyComments = buf_dicom.StudyComments
    
    buffer_row = {
        'patient_uid': patient_id,
        'sex': patient_gender,
        'date_of_birth': patient_date_of_birth, # should be obtained from RIS file
        'patient_access_number_id': patient_access_number,
        'studies': {'StudyID': StudyID if not len(StudyID) == 0 else np.nan,
                    'StudyDate': StudyDate if not len(StudyDate) == 0 else np.nan,
                    'StudyTime': StudyTime if not len(StudyTime) == 0 else np.nan,
                    'StudyDescription': StudyDescription if not len(StudyDescription) == 0 else np.nan,
                    'StudyInstanceUID': StudyInstanceUID if not len(StudyInstanceUID) == 0 else np.nan,
                    'StudyStatusID': StudyStatusID if not len(StudyStatusID) == 0 else np.nan,
                    'StudyComments': StudyComments if not len(StudyComments) == 0 else np.nan,
                    }
    }
    
    rows.append(buffer_row)

    
# pass dict with all records into pandas DataFrame    
df = pd.DataFrame.from_dict(rows)

# # REPAIR: dropping equal values
# drop these rows because:
# 1. Yes, they do not have actual id to compare (another form)
# 2. But, found, that such records which have broken format,
# they have exactly the same "accession_number_id" as one
# row above or below record <- that allowed cauth insight 
# for duplicating values. (Better to shuffle data!!!).

# for index, id in enumerate(df['patient_uid']):
#     if not id.startswith('1.2'):
#         df.drop([index], axis=0, inplace=True)



# # # REPAIR: dropping equal values because all values in row are equal, including values in stuides
# df.drop_duplicates(subset='patient_uid', ignore_index=True, inplace=True)

# df

  0%|          | 0/904 [00:00<?, ?it/s]/Users/macbook/miniconda3/envs/ML/lib/python3.6/site-packages/pydicom/jsonrep.py:130: UserWarning: no bulk data URI handler provided for retrieval of value of data element "7FE00010"
  'of value of data element "{}"'.format(self.tag)
100%|██████████| 904/904 [00:02<00:00, 369.56it/s]


In [ ]:
# RIS data <-- radiologist opinions on a case
# - pat_id - non unique
# - id - UNIQUE* ('id' maps with 'accession_number_id' from PAC data)

# PAC data <-- all the imaging data from the PACS, one line per image
# - patient_uid - non unique
# - patient_access_number_id - non unique 
# but if drop duplicates by "patient_access_number_id",
# then get desired number == 248 rows (UNIQUE*) 

In [819]:
# RIS
# renamed column ID --> patient_access_number_id <-- because it 
# is mapping column between PAC AND RIS DATA. {Number of rows equal 248 == 248}

ris_df

,pat_id,pat_sex,pat_dob,id,outcome_l,outcome_r,date
0,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,R2,R2,20140808
1,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,7134ddfd-f53b-4ab1-a2c7-2b0b56c574d2,R1,R3,20161214
2,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,71ad015d-b14c-45d1-8508-ea9dca9b0a9a,R1,R3,20180521
3,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,0fc40be6-9adc-407e-b762-0cc3341360b7,R1,R2,20200725
4,bf98f2e9-f02e-48b0-8472-470af8aebf7a,F,19620722,47821fb9-6aa6-44c2-9c8d-6adb6ea24251,R2,R2,20221030
...,...,...,...,...,...,...,...
243,55e8e3d4-a3be-4a91-a880-a4497e130c94,F,19700722,fe76e9da-435d-4327-945d-3edcb442cbf8,R2,R1,20220103
244,637f4e3c-b786-4229-a8f3-b2bd91e97265,F,19620722,0832e6de-2fc7-4655-9d58-1bff0e18b23d,R2,R3,20120824
245,f5c8540f-c885-40ad-9081-1763e45fe828,F,19680722,23f32605-7bf6-4505-bd31-f45d54f4917f,R1,R3,20210816
246,f5c8540f-c885-40ad-9081-1763e45fe828,F,19680722,f44ed697-4fe3-4573-b967-c66f61171d5a,R2,R1,20230711


In [824]:
# PAC
# REPAIR: we change "date_of_birth" in PAC on values from RIS  

#df.drop_duplicates(subset='patient_access_number_id', ignore_index=True)

df


,patient_uid,sex,date_of_birth,patient_access_number_id,studies
0,1.2.826.0.1.10184103.5.1.80750,F,2014.1.8,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,"{'StudyID': nan, 'StudyDate': '20221030', 'Stu..."
1,1.2.826.0.1.10184103.5.1.80750,F,2014.1.8,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,"{'StudyID': nan, 'StudyDate': '20221030', 'Stu..."
2,1.2.826.0.1.10184103.5.1.80750,F,2014.1.8,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,"{'StudyID': nan, 'StudyDate': '20221030', 'Stu..."
3,1.2.826.0.1.10184103.5.1.80750,F,2014.1.8,41e1dea8-0c5d-4ca1-9fcd-041c67b2d1c0,"{'StudyID': nan, 'StudyDate': '20221030', 'Stu..."
4,1.2.826.0.1.10184103.5.1.148327,F,2014.1.8,7134ddfd-f53b-4ab1-a2c7-2b0b56c574d2,"{'StudyID': nan, 'StudyDate': '20221030', 'Stu..."
...,...,...,...,...,...
899,1.2.826.0.1.10184103.5.1.34358,F,2014.1.8,f44ed697-4fe3-4573-b967-c66f61171d5a,"{'StudyID': nan, 'StudyDate': '20250329', 'Stu..."
900,f5c8540f-c885-40ad-9081-1763e45fe828,F,2014.1.8,5cd192a0-fcb0-4fb8-8f29-a5f88afa7256,"{'StudyID': nan, 'StudyDate': '20250329', 'Stu..."
901,1.2.826.0.1.10184103.5.1.93121,F,2014.1.8,5cd192a0-fcb0-4fb8-8f29-a5f88afa7256,"{'StudyID': nan, 'StudyDate': '20250329', 'Stu..."
902,f5c8540f-c885-40ad-9081-1763e45fe828,F,2014.1.8,5cd192a0-fcb0-4fb8-8f29-a5f88afa7256,"{'StudyID': nan, 'StudyDate': '20250329', 'Stu..."


In [ ]:
for patient_index, patient in enumerate(tqdm(pac_data)):
    buf_dicom = Dataset.from_json(patient)

In [818]:
buf_dicom

(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: Digital Mammography X-Ray Image Storage - For Presentation
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.826.0.1.10184103.5.3.1059555
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])
(0002, 0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.1
(0002, 0013) Implementation Version Name         SH: 'OFFIS_DCMTK_361'
(0002, 0016) Source Application Entity Title     AE: 'RR8WFMKHN'
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', None]
(0008, 0016) SOP Class UID                       UI: Digital Mammography X-Ray Image Storage - For Presentation
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.10184103.5.3.1059555
(0008, 0020) Study Date        